In [1]:
import requests
import json
import pymongo
import pandas as pd

In [10]:
# Load Kaggle CSV and drop duplicates

kaggle_df = pd.read_csv('Datafiniti_Fast_Food_Restaurants.csv')
kaggle_df = kaggle_df.drop(columns=['categories', 'keys','sourceURLs', 'websites']).drop_duplicates()
kaggle_df = kaggle_df.loc[kaggle_df['province'].isin(['IL', 'MI', 'IN'])]
#kaggle_df = kaggle_df.set_index('id', drop=True)
#test_df = kaggle_df.head(5)
#test_df

In [11]:
# Set up Yelp API request
from config import yelpkey

base_url = "https://api.yelp.com/v3/businesses/search"
headers = {'Authorization': f'Bearer {yelpkey}'}

# Create connection to Mongo Database
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.fastfood_db

In [14]:
row_number = 0
db['yelp'].drop()

for index, row in kaggle_df.iterrows():
    row_number += 1
    params = {
        'term': row['name'],
        'latitude': row['latitude'],
        'longitude': row['longitude'],
        'limit': 1
    }    
    try:
        response = requests.get(base_url, params=params, headers=headers).json()['businesses'][0]
        response['kaggle_id'] = row['id']
        db.yelp.insert_one(response)
        print(f'Processing record {row_number}')
    except KeyError:
        print(f'No result for record {row_number} [KeyError]')
    except IndexError:
        print(f'No result for record {row_number} [IndexError]')

Processing record 1
Processing record 2
Processing record 3
Processing record 4
Processing record 5
Processing record 6
Processing record 7
Processing record 8
Processing record 9
Processing record 10
Processing record 11
Processing record 12
Processing record 13
Processing record 14
Processing record 15
Processing record 16
Processing record 17
Processing record 18
Processing record 19
Processing record 20
Processing record 21
Processing record 22
Processing record 23
Processing record 24
Processing record 25
Processing record 26
Processing record 27
Processing record 28
Processing record 29
Processing record 30
Processing record 31
Processing record 32
Processing record 33
Processing record 34
Processing record 35
Processing record 36
Processing record 37
Processing record 38
Processing record 39
Processing record 40
Processing record 41
Processing record 42
Processing record 43
Processing record 44
Processing record 45
Processing record 46
Processing record 47
Processing record 48
P

Processing record 379
Processing record 380
Processing record 381
Processing record 382
Processing record 383
Processing record 384
Processing record 385
Processing record 386
Processing record 387
Processing record 388
Processing record 389
Processing record 390
Processing record 391
Processing record 392
Processing record 393
Processing record 394
Processing record 395
Processing record 396
Processing record 397
Processing record 398
Processing record 399
Processing record 400
Processing record 401
Processing record 402
Processing record 403
Processing record 404
Processing record 405
No result for record 406 [IndexError]
Processing record 407
Processing record 408
Processing record 409
Processing record 410
Processing record 411
Processing record 412
Processing record 413
Processing record 414
Processing record 415
Processing record 416
Processing record 417
Processing record 418
Processing record 419
Processing record 420
Processing record 421
Processing record 422
Processing recor

Processing record 751
Processing record 752
Processing record 753
Processing record 754
Processing record 755
Processing record 756
Processing record 757
Processing record 758
Processing record 759
Processing record 760
Processing record 761
Processing record 762
Processing record 763
Processing record 764
Processing record 765
Processing record 766
Processing record 767
Processing record 768
Processing record 769
Processing record 770
Processing record 771
Processing record 772
Processing record 773
Processing record 774
Processing record 775
Processing record 776
Processing record 777
Processing record 778
Processing record 779
Processing record 780
Processing record 781
Processing record 782
Processing record 783
Processing record 784
Processing record 785
Processing record 786
Processing record 787
Processing record 788
Processing record 789
Processing record 790
Processing record 791
Processing record 792
Processing record 793
Processing record 794
Processing record 795
Processing

In [23]:
# By moving longitude and latitude from the array 'coordinates', we save ourselves a headache when moving these documents into pandas

yelp_locations = list(db.yelp.find())
for yelp_location in yelp_locations:
    myquery = {'id': yelp_location['id']}
    newvalues = {'$set': {'latitude': yelp_location['coordinates']['latitude'],
                          'longitude': yelp_location['coordinates']['longitude']}}
    db.yelp.update_one(myquery, newvalues)


In [27]:
db.yelp.find_one({'id': yelp_location['id']})

{'_id': ObjectId('5cd7107afd64d7121c37ed3d'),
 'id': 'ZSj-KCzDSQaMPgMCNnCDhg',
 'alias': 'wendys-detroit-3',
 'name': "Wendy's",
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/8i8vRgx42X3j5ViHDhq2yA/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/wendys-detroit-3?adjust_creative=e1zDuraqPYNqfEmgXJJAyQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=e1zDuraqPYNqfEmgXJJAyQ',
 'review_count': 2,
 'categories': [{'alias': 'hotdogs', 'title': 'Fast Food'},
  {'alias': 'burgers', 'title': 'Burgers'}],
 'rating': 2.0,
 'coordinates': {'latitude': 42.3687210083008, 'longitude': -83.1380462646484},
 'transactions': [],
 'price': '$',
 'location': {'address1': '9768 Grand River Ave',
  'address2': '',
  'address3': '',
  'city': 'Detroit',
  'zip_code': '48204',
  'country': 'US',
  'state': 'MI',
  'display_address': ['9768 Grand River Ave', 'Detroit, MI 48204']},
 'phone': '+13139315958',
 'display_phone': '(313) 931-5958',
 'distance': 5.1303142330